**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("titanic.zip"), directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Predspracovanie dát

V predchádzajúcich príkladoch sme pracovali s dátovou množinou Iris. Keďže táto dátová množina obsahuje len 4 stĺpce a dáta v nich sú numerické a s približne rovnakou škálou, nebolo ich potrebné žiadnym zvláštnym spôsobom predspracovať.

V praxi sa takéto prípady vyskytujú veľmi zriedkakedy. Príprava dátovej množiny väčšinou zaberie omnoho viac vývojárskeho času než aplikácia a ladenie samotného modelu. Dáta treba typicky vyčistiť, ošetriť chýbajúce hodnoty, vhodne preškálovať, v niektorých prípadoch je potrebné prekódovať kategorické premenné a pod.

V tomto notebook-u si ukážeme, ako fungujú niektoré základné typy predspracovania a ako sa dá fáza predspracovania navrhnúť tak, aby sa dal celý postup ľahko reprodukovať pre nové dáta.

Celkový postup pri tréningu modelu sa dá zhrnúť takto:

* Načítame dátovú množinu.
* Oddelíme tréningové a testovacie dáta (ak ešte nie sú rozdelené).
* Vyčístíme a predspracujeme ju, napr.:* Ošetríme chýbajúce hodnoty.
* Preškálujeme numerické dáta do vhodných rozsahov.
* Prekódujeme kategorické premenné z textovej do číselnej reprezentácie.

* Natrénujeme model na tréningových dátach.
* Otestujeme zovšeobecnenie na testovacích dátach.
### Načítanie dátovej množiny Titanic

Ako príklad na predspracovanie použijeme známu dátovú množinu [Titanic](https://www.kaggle.com/c/titanic). Dátová množina obsahuje údaje o pasažieroch z Titanicu. Úlohou je predikovať, ktorí haváriu prežili a ktorí nie. Aby sme získali predstavu, s akými údajmi budeme približne pracovať, zobrazme si najprv stručný opis dát zo súboru `description.txt`:



In [ ]:
with open("data/titanic/description", "r") as file:
    print("".join(file.readlines()))

Ďalej načítajme z CSV súboru samotnú dátovú množinu a rozdeľme si ju na tréningovú a testovaciu časť. Stratifikujeme podľa triedy (či sa pasažier zachránil alebo nie):



In [ ]:
df = pd.read_csv("data/titanic/train.csv")
df_train, df_test = train_test_split(df, test_size=0.25,
                     stratify=df["Survived"], random_state=4)

### Jednoduché predspracovanie

#### Škálovanie numerických vstupov

Pre mnohé metódy strojového učenia je vhodné numerické dáta najprv preškálovať do nejakého štandardného rozsahu – inak môže metóda dátam, ktoré majú väčšiu relatívnu škálu, prikladať väčšiu váhu, čo typicky nie je žiaduce. Existuje viacero typov takého škálovania, často sa používa napríklad:

* Škálovanie do rozsahu od 0 po 1 (dá sa použiť `sklearn.preprocessing.MinMaxScaler`);
* Štandardizácia (stredná hodnota sa posunie na nulu a rozptyl preškáluje na 1; `sklearn.preprocessing.StandardScaler`);
* ...
Ukážme si príklad štandardizácie (iné typy škálovania sa používajú obdobným spôsobom) – dajme tomu, že chceme štandardizovať stĺpec `Fare`. V rámci balíčka `scikit-learn`, s ktorým budeme pracovať, sa všetky podobné operácie realizujú pomocou štandardného rozhrania – tzv. transformátorov. Každý transformátor sa najprv skonštruuje a potom sa dá naladiť podľa dát pomocou metódy `fit`:



In [ ]:
scaler = StandardScaler()
scaler.fit(df_train[['Fare']])

Transformované dáta je možné získať pomocou metódy `transform`:



In [ ]:
fare_scaled = scaler.transform(df_train[['Fare']])

Ak chceme dáta použiť na naladenie transformátora, ale zároveň chceme tie isté dáta aj preškálovať, je k dispozícii aj kombinovaná funkcia `fit_transform` – v našom prípade bude teda lepšie použiť tú:



In [ ]:
scaler = StandardScaler()
fare_scaled = scaler.fit_transform(df_train[['Fare']])

Zobrazme si teraz časť pôvodného a transformovaného stĺpca, aby sme videli, či náš transformátor funguje:



In [ ]:
pd.DataFrame(
    np.hstack([df_train[['Fare']], fare_scaled]),
    columns=["Fare", "Fare Scaled"]
).head()

Môžeme sa tiež pozrieť na stredné hodnoty a rozptyly, aby sme zistili, či sa naozaj zmenili tak, ako sme predpokladali:



In [ ]:
print("Mean of fare: {}\nVariance of fare: {}".format(
    np.mean(df_train['Fare']),
    np.var(df_train['Fare'])
))

In [ ]:
print("Mean of scaled fare: {}\nVariance of scaled fare: {}".format(
    np.mean(fare_scaled),
    np.var(fare_scaled)
))

#### Prekódovanie kategorických premenných

V dátových množinách sa často vyskytujú kategorické premenné, ktoré nadobúdajú určitý pomerne malý počet diskrétnych hodnôt, reprezentovaných textovými reťazcami. V našom prípade patria medzi takéto premenné napr. `Embarked` (prístav, kde cestujúci nastúpil na loď) a `Sex` (pohlavie cestujúceho).

Takéto premenné môže byť (v závislosti od použitej metódy) potrebné transformovať z textových reťazcov na numerické identifikátory (každej textovej hodnote sa priradí nejaké číslo). V Python-e to je možné realizovať úplne obdobne ako preškálovanie numerického atribútu – iba sa použije iný transformátor: `OrdinalEncoder`.



In [ ]:
ordenc = OrdinalEncoder()
sex_encoded = ordenc.fit_transform(df_train[["Sex"]])

Znovu si pre porovnanie zobrame pôvodný aj prekódovaný stĺpec:



In [ ]:
pd.DataFrame(
    np.hstack([df_train[["Sex"]], sex_encoded]),
    columns=["Sex", "Sex Encoded"]
).head()

Ako vidno, pohlavie `female` bolo prekódované ako 0 a pohlavie `male` ako 1. Overiť si to môžeme aj podľa poradia jednotlivých označení v nasledujúcom zozname:



In [ ]:
ordenc.categories_

### Ošetrenie chýbajúcich hodnôt

Keď sa pokúsime vyššie uvedené prístupy aplikovať na ďalšie stĺpce z dátovej množiny, zistíme, že to nie vždy funguje. Pri dátových množinách sa totiž často stáva, že niektoré údaje chýbajú (v niektorých riadkoch nie sú vyplnené všetky stĺpce). Pre väčšinu metód strojového učenia to predstavuje problém a chýbajúce hodnoty treba nejakým spôsobom ošetriť. Existujú v zásade tri skupiny prístupov ako sa s chýbajúcimi hodnotami vysporiadať:

* Príslušné riadky z dátovej množiny vypustiť.
* Chýbajúce hodnoty podľa nejakého pravidla doplniť (angl. imputation).
* Chýbajúce hodnoty ponechať, ak sa s nimi vie vysporiadať priamo metóda strojového učenia (napr. niektoré implementácie rozhodovacích stromov).
Úplné vypustenie riadkov sa robí väčšinou v prípade, keď sa dá predpokladať, že riadok obsahuje málo užitočných informácií (napr. chýbajú skoro všetky hodnoty) alebo keď je dát také veľké množstvo, že neúplné záznamy nie je vôbec potrebné použiť (čo sa stáva zriedkavo).

Doplnenie chýbajúcich hodnôt môže byť rôzne zložité:

* Veľmi jednoduché – napr. chýbajúce hodnoty sa doplnia priernou alebo najčastejšou hodnotou z daného stĺpca.
* Veľmi zložité – napr. sa na iných stĺpcoch natrénuje celý model a ten predikuje chýbajúce hodnoty.
* Stredne zložité...
My si ukážeme len jeden triviálny spôsob doplnenia pre numerické a jeden pre kategorické dáta (v oboch prípadoch použijeme triedu `SimpleImputer`), ale v balíčku `scikit-learn` a inde sa dajú nájsť aj ďalšie (napr. `sklearn.impute.IterativeImputer`).

#### Detekcia chýbajúcich hodnôt

Predtým, ako prejdeme ku samotnému dopĺňaniu chýbajúcich hodnôt, ukážme, ako sa dajú chýbajúce hodnoty detegovať. Balíček `pandas` má na ten účel fukciu `.isnull()`, ktorá nám pre každú bunku vráti či v nej chýba hodnota alebo nie:



In [ ]:
df_train["Age"].isnull()[:10]

Ak chceme vedieť, či v danom stĺpci chýba aspoň jedna hodnota, môžeme zreťaziť funkciu `.isnull()` s funkciou `.any()`:



In [ ]:
df_train["Age"].isnull().any()

In [ ]:
df_train["Fare"].isnull().any()

As we can see, some values are missing in the `Age` column, but in the `Fare` column we have values for all passengers.

#### Trivial Imputation of Numeric Values

A trivial way to impute missing numeric values is to use the `SimpleImputer` transformer. This transformer will replace missing values with the average for that column by default. It is, however, possible to use different strategies as well:



In [ ]:
num_impute = SimpleImputer()
age_imputed = num_impute.fit_transform(df_train[["Age"]])

Výsledok bude vyzerať nasledovne:



In [ ]:
pd.DataFrame(
    np.hstack([df_train[['Age']], age_imputed]),
    columns=["Age", "Age Imputed"]
).head()

#### Doplnenie kategorických atribútov

Na doplnenie chýbajúcich hodnôt pre kategorické atribúty môžeme znovu použiť transformátor `SimpleImputer` – budeme ho len odlišne parametrizovať. Ak chceme napríklad chýbajúce hodnoty doplniť najčastejšou hodnotou atribútu:



In [ ]:
cat_impute = SimpleImputer(strategy="most_frequent")
embarked_imputed = cat_impute.fit_transform(df_train[["Embarked"]])

Alternatívne môžeme kategorickému atribútu pridať novú hodnotu, ktorú nazveme `MISSING`. Tá bude indikovať, že hodnota chýbala:



In [ ]:
cat_impute = SimpleImputer(strategy='constant', fill_value='MISSING')
embarked_imputed = cat_impute.fit_transform(df_train[["Embarked"]])

### Opakovateľnosť predspracovania

Prirodzene, že keď navrhneme fázu predspracovania, boli by sme radi, keby sme ju mohli následne rovnako aplikovať aj na testovacie dáta a neskôr, po nasadení modelu, na všetky nové dáta. Jedna vec, na ktorú si pritom treba dať pozor je, že parametre transformátorov ladíme na dátach. Na predspracovanie testovacích a ďalších dát musíme použiť rovnako naladené transformátory, inak dostaneme iné výsledky a náš model nebude správne fungovať. Mohlo by sa napríklad stať, že tá istá kategorická hodnota sa v rámci trénovacej množiny prekóduje ako 3 a v rámci testovacej ako 1.

#### Nesprávny spôsob predspracovania

Ukážme si malý príklad nesprávneho prípadu predspracovania. Dajme tomu, že autor kódu chcel dosiahnuť opakovateľnosť a preto kód na predspracovanie obalil do funkcie, ktorú volá najprv z tréningovými a neskôr s testovacími dátami. Zabudol však, že vnútri funkcie sa ladia parametre transformátorov, ktoré sa naladia zakaždým inak.



In [ ]:
def preprocess(df):
    num_impute = SimpleImputer()
    age_imputed = num_impute.fit_transform(df[['Age']])
    
    scaler = StandardScaler()
    age_scaled = scaler.fit_transform(age_imputed)
    
    return age_scaled

In [ ]:
df_train_preproc = preprocess(df_train)
df_test_preproc = preprocess(df_test)

Výstup bude vyzerať nasledovne. Malo by byť z neho vidno, že hodnota 26 sa v každom prípade prekódovala na iné číslo, čo je samozrejme neprijateľné.



In [ ]:
pd.DataFrame(
    np.hstack([df_train[['Age']], df_train_preproc]),
    columns=["Age", "Scaled Age"]
).head()

In [ ]:
pd.DataFrame(
    np.hstack([df_test[['Age']], df_test_preproc]),
    columns=["Age", "Scaled Age"]
).head()

#### Správny postup

Správne sa musí ten istý transformátor, naladený na tréningových dátach, aplikovať aj na testovacie dáta. Dalo by sa to realizovať napríklad takto:



In [ ]:
def preprocess(df, params=None):
    if params is None:
        params = {}
        
        params["num_impute"] = SimpleImputer()
        age_imputed = params["num_impute"].fit_transform(df[['Age']])
        
        params["scaler"] = StandardScaler()
        age_scaled = params["scaler"].fit_transform(age_imputed)
        
    else:
        age_imputed = params["num_impute"].transform(df[['Age']])
        age_scaled = params["scaler"].transform(age_imputed)
        
    return age_scaled, params

In [ ]:
df_train_preproc, params = preprocess(df_train)
df_test_preproc, params = preprocess(df_test, params)

Keďže oba transformátory sme si tento raz uložili a na testovacie dáta aplikovali už len pomocou funkcie `transform`, výsledky by tento raz mali byť správne.



In [ ]:
pd.DataFrame(
    np.hstack([df_train[['Age']], df_train_preproc]),
    columns=["Age", "Scaled Age"]
).head()

In [ ]:
pd.DataFrame(
    np.hstack([df_test[['Age']], df_test_preproc]),
    columns=["Age", "Scaled Age"]
).head()

#### Jednoduchší prístup: scikit-learn pipelines

Problém takéhoto prístupu ku predspracovaniu je, že v praxi je predspracovanie často dosť komplikované. Uchovávať si manuálne všetky transformátory, ktoré boli v rámci neho použité, a následne ich opätovne rovnakým spôsobom aplikovať je pomerne prácná úloha – a dá sa pri nej ľahko pomýliť. Preto si v ďalšom notebook-u ukážeme ako sa dá tento postup automatizovať pomocou konceptu tzv. **pipelines**  – tiež z balíčka `scikit-learn`.

